# 영화 리뷰(IMDB) 이진 분류

# IMDB 데이터 셋
영화 리뷰에 대한 데이터 50000개로 구성. 이중 25,000개의 훈련데이터와 25,000개의 테스트 데이터로 나눠짐.
이미 전처리가 되어 있으며 각 리뷰가 숫자로 변환되어 있음
원리는 사전에 있는 단어와 숫자를 매핑시켜서 단어대신 숫자로 표현한 것임.

In [1]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 2s 0us/step


**num_words=10000** 은 훈련데이터에서 가장 자주 사용하는 단어 1만개만 사용하겠다는 의미
사용 빈도수가 10000등 안에 들지 못하는 데이터는 training 시키지 않습니다.

train_data는 각 단어에 매핑되는 숫자로 전처리 되어 있고
train_labels는 긍정(1), 부정(0)을 의미

In [8]:
train_data[0]   # [1,14,22, ..., 19, 178, 32]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [5]:
train_labels[0] # 

1

## 리뷰 데이터 복구
현재 숫자로 전처리 되어있는 데이터를 다시 영어로 복구

In [20]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

decode_review = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]])

In [18]:
decode_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [21]:
word_index = imdb.get_word_index()

In [22]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

train_data을 디코딩하는데 get()은 key값을 통해 value를 찾아주고 못찾은 경우에는 '?'로 대체합니다.
이 작업이 필요한 이유는 위에서 데이터를 받아올 때 num_words=10000을 통해 만개의 자주 쓰이는 단어만 가져왔으므로 나머지 단어에 대해서는 값이 존재하지 않으므로 다음 작업이 필요합니다. 그리고 각 단어를 join()을 통해서 공백으로 이어주었습니다. 이때 0,1,2에 해당하는 '패딩', '문서시작','사전에 없음'을 위함이므로 3을 빼고 계산합니다.

In [23]:
decode_review = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]])

## 데이터 준비
- 신경망에 현재의 데이터 형태인 숫자 리스트를 대입할 수 없습니다. 따라서 리스트를 텐서로 바꿔줘야 합니다.
그 과정에서 원 핫 인코딩 방식을 사용했습니다.
- len(sequences) * 10000 크기의 성분이 0인 2차원 넘파이 배열을 만들어 줌

In [48]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        # print(i, sequence)              # 확인 코드
        results[i, sequence] = 1. 
    return results 

In [51]:
# 확인 코드
aa = np.array([[1,2,3], [4,5,6]])
vectorize_sequences(aa)

0 [1 2 3]
1 [4 5 6]


array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

- 넘파이 배열에만 있는 방법인데 2차원 배열 results에 대해서 i번째 행에 Sequence리스트의 원소를 index로 하여 접근
- 그 자리에 1을 대입하여 원 핫 인코딩을 실행
- 각 데이터를 텐서로 변경 

In [30]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)


- 레이블도 벡터로 변경

In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [27]:
print(train_data[1])

[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 8163, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 6853, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 8255, 2, 349, 2637, 148, 605, 2, 8003, 15, 123, 125, 68, 2, 6853, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 8255, 5, 2, 656, 245, 2350, 5, 4, 9837, 131, 152, 491, 18, 2, 32, 7464, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]


In [31]:
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [32]:
x_train.shape

(25000, 10000)